Import of all necessary libraries

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from nltk.tokenize import TweetTokenizer
from scipy.sparse import hstack
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})


In [ ]:
#@title Function for preprocessing
def clean_dataframe(df, ohe_date=None):
  # FLAG è uguale per tutti
    df_new = df.drop(columns=['flag'])

    substitutions = {
        '&quot;': '"',
        '&amp;': '&',
        '&gt;': '>',
        '&lt;': '<',
        '<3': 'love'
        }
    
    for abb, description in substitutions.items():
        df_new['text'] = df_new['text'].str.replace(abb, description, regex=False)
        
    
    tweet_length = df['text'].str.len()
    
    # Removing links
    regex = '(www\.)|(https?:\/\/)[-a-zA-Z0-9@:%._\+~#=\/]*'
    contains_links = df_new['text'].str.contains(regex)
    df_new['text'] = df_new['text'].str.replace(regex,'',regex=True)

    # removing hashtags
    regex='#\w+'
    df_new['text'] = df_new['text'].str.replace(regex,'',regex=True)
    
    # count number of mentions
    regex = '@\w+' 
    contains_mentions = df_new['text'].str.contains(regex)
    
  
    EMOTICONS_EMO = {
        u":‑)":"Happy face or smiley",
        u":-)))":"Very very Happy face or smiley",
        u":)":"Happy face or smiley",
        u":))":"Very Happy face or smiley",
        u":)))":"Very very Happy face or smiley",
        u":-]":"Happy face or smiley",
        u":]":"Happy face or smiley",
        u":-3":"Happy face smiley",
        u":3":"Happy face smiley",
        u":->":"Happy face smiley",
        u":>":"Happy face smiley",
        u"8-)":"Happy face smiley",
        u":o)":"Happy face smiley",
        u":-}":"Happy face smiley",
        u":}":"Happy face smiley",
        u":-)":"Happy face smiley",
        u":c)":"Happy face smiley",
        u":^)":"Happy face smiley",
        u"=]":"Happy face smiley",
        u"=)":"Happy face smiley",
        u":‑D":"Laughing, big grin or laugh with glasses",
        u":D":"Laughing, big grin or laugh with glasses",
        u"8‑D":"Laughing, big grin or laugh with glasses",
        u"8D":"Laughing, big grin or laugh with glasses",
        u"X‑D":"Laughing, big grin or laugh with glasses",
        u"XD":"Laughing, big grin or laugh with glasses",
        u"=D":"Laughing, big grin or laugh with glasses",
        u"=3":"Laughing, big grin or laugh with glasses",
        u"B^D":"Laughing, big grin or laugh with glasses",
        u":-))":"Very happy",
        u":-(":"Frown, sad, andry or pouting",
        u":‑(":"Frown, sad, andry or pouting",
        u":(":"Frown, sad, andry or pouting",
        u":‑c":"Frown, sad, andry or pouting",
        u":c":"Frown, sad, andry or pouting",
        u":‑<":"Frown, sad, andry or pouting",
        u":<":"Frown, sad, andry or pouting",
        u":‑[":"Frown, sad, andry or pouting",
        u":[":"Frown, sad, andry or pouting",
        u":-||":"Frown, sad, andry or pouting",
        u">:[":"Frown, sad, andry or pouting",
        u":{":"Frown, sad, andry or pouting",
        u":@":"Frown, sad, andry or pouting",
        u">:(":"Frown, sad, andry or pouting",
        u":'‑(":"Crying",
        u":'(":"Crying",
        u":'‑)":"Tears of happiness",
        u":')":"Tears of happiness",
        u"D‑':":"Horror",
        u"D:<":"Disgust",
        u"D:":"Sadness",
        u"D8":"Great dismay",
        u"D;":"Great dismay",
        u"D=":"Great dismay",
        u"DX":"Great dismay",
        u":‑O":"Surprise",
        u":O":"Surprise",
        u":‑o":"Surprise",
        u":o":"Surprise",
        u":-0":"Shock",
        u"8‑0":"Yawn",
        u">:O":"Yawn",
        u":-*":"Kiss",
        u":*":"Kiss",
        u":X":"Kiss",
        u";‑)":"Wink or smirk",
        u";)":"Wink or smirk",
        u"*-)":"Wink or smirk",
        u"*)":"Wink or smirk",
        u";‑]":"Wink or smirk",
        u";]":"Wink or smirk",
        u";^)":"Wink or smirk",
        u":‑,":"Wink or smirk",
        u";D":"Wink or smirk",
        u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
        u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
        u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
        u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
        u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
        u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
        u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
        u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
        u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
        u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
        u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
        u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
        u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
        u">:[(\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
        u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
        u":[(\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
        u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
        u"=[(\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
        u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
        u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
        u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
        u":‑|":"Straight face",
        u":|":"Straight face",
        u":$":"Embarrassed or blushing",
        u":‑x":"Sealed lips or wearing braces or tongue-tied",
        u":x":"Sealed lips or wearing braces or tongue-tied",
        u":‑#":"Sealed lips or wearing braces or tongue-tied",
        u":#":"Sealed lips or wearing braces or tongue-tied",
        u":‑&":"Sealed lips or wearing braces or tongue-tied",
        u":&":"Sealed lips or wearing braces or tongue-tied",
        u"O:‑)":"Angel, saint or innocent",
        u"O:)":"Angel, saint or innocent",
        u"0:‑3":"Angel, saint or innocent",
        u"0:3":"Angel, saint or innocent",
        u"0:‑)":"Angel, saint or innocent",
        u"0:)":"Angel, saint or innocent",
        u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
        u"0;^)":"Angel, saint or innocent",
        u">:‑)":"Evil or devilish",
        u">:)":"Evil or devilish",
        u"}:‑)":"Evil or devilish",
        u"}:)":"Evil or devilish",
        u"3:‑)":"Evil or devilish",
        u"3:)":"Evil or devilish",
        u">;)":"Evil or devilish",
        u"|;‑)":"Cool",
        u"|‑O":"Bored",
        u":‑J":"Tongue-in-cheek",
        u"#‑)":"Party all night",
        u"%‑)":"Drunk or confused",
        u"%)":"Drunk or confused",
        u":-###..":"Being sick",
        u":###..":"Being sick",
        u"<:‑|":"Dump",
        u"(>_<)":"Troubled",
        u"(>_<)>":"Troubled",
        u"(';')":"Baby",
        u"(^^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
        u"(^_^;)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
        u"(-_-;)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
        u"(~_~;) (・.・;)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
        u"(-_-)zzz":"Sleeping",
        u"(^_-)":"Wink",
        u"((+_+))":"Confused",
        u"(+o+)":"Confused",
        u"(o|o)":"Ultraman",
        u"^_^":"Joyful",
        u"(^_^)/":"Joyful",
        u"(^O^)／":"Joyful",
        u"(^o^)／":"Joyful",
        u"(__)":"Kowtow as a sign of respect, or dogeza for apology",
        u"_(._.)_":"Kowtow as a sign of respect, or dogeza for apology",
        u"<(_ _)>":"Kowtow as a sign of respect, or dogeza for apology",
        u"<m(__)m>":"Kowtow as a sign of respect, or dogeza for apology",
        u"m(__)m":"Kowtow as a sign of respect, or dogeza for apology",
        u"m(_ _)m":"Kowtow as a sign of respect, or dogeza for apology",
        u"('_')":"Sad or Crying",
        u"(/_;)":"Sad or Crying",
        u"(T_T) (;_;)":"Sad or Crying",
        u"(;_;":"Sad of Crying",
        u"(;_:)":"Sad or Crying",
        u"(;O;)":"Sad or Crying",
        u"(:_;)":"Sad or Crying",
        u"(ToT)":"Sad or Crying",
        u";_;":"Sad or Crying",
        u";-;":"Sad or Crying",
        u";n;":"Sad or Crying",
        u";;":"Sad or Crying",
        u"Q.Q":"Sad or Crying",
        u"T.T":"Sad or Crying",
        u"QQ":"Sad or Crying",
        u"Q_Q":"Sad or Crying",
        u"(-.-)":"Shame",
        u"(-_-)":"Shame",
        u"(一一)":"Shame",
        u"(；一_一)":"Shame",
        u"(=_=)":"Tired",
        u"(=^·^=)":"cat",
        u"(=^··^=)":"cat",
        u"=_^= ":"cat",
        u"(..)":"Looking down",
        u"(._.)":"Looking down",
        u"^m^":"Giggling with hand covering mouth",
        u"(・・?":"Confusion",
        u"(?_?)":"Confusion",
        u">^_^<":"Normal Laugh",
        u"<^!^>":"Normal Laugh",
        u"^/^":"Normal Laugh",
        u"（*^_^*）" :"Normal Laugh",
        u"(^<^) (^.^)":"Normal Laugh",
        u"(^^)":"Normal Laugh",
        u"(^.^)":"Normal Laugh",
        u"(^_^.)":"Normal Laugh",
        u"(^_^)":"Normal Laugh",
        u"(^^)":"Normal Laugh",
        u"(^J^)":"Normal Laugh",
        u"(*^.^*)":"Normal Laugh",
        u"(^—^）":"Normal Laugh",
        u"(#^.^#)":"Normal Laugh",
        u"（^—^）":"Waving",
        u"(;_;)/~~~":"Waving",
        u"(^.^)/~~~":"Waving",
        u"(-_-)/~~~ ($··)/~~~":"Waving",
        u"(T_T)/~~~":"Waving",
        u"(ToT)/~~~":"Waving",
        u"(*^0^*)":"Excited",
        u"(*_*)":"Amazed",
        u"(*_*;":"Amazed",
        u"(+_+) (@_@)":"Amazed",
        u"(*^^)v":"Laughing,Cheerful",
        u"(^_^)v":"Laughing,Cheerful",
        u"((d[-_-]b))":"Headphones,Listening to music",
        u'(-"-)':"Worried",
        u"(ーー;)":"Worried",
        u"(^0_0^)":"Eyeglasses",
        u"(＾ｖ＾)":"Happy",
        u"(＾ｕ＾)":"Happy",
        u"(^)o(^)":"Happy",
        u"(^O^)":"Happy",
        u"(^o^)":"Happy",
        u")^o^(":"Happy",
        u":O o_O":"Surprised",
        u"o_0":"Surprised",
        u"o.O":"Surpised",
        u"(o.o)":"Surprised",
        u"oO":"Surprised",
        u"(*￣m￣)":"Dissatisfied",
        u"(‘A`)":"Snubbed or Deflated"
    }
    
    print("Traduco emoticons")
    for abb, description in EMOTICONS_EMO.items():
        df_new['text'] = df_new['text'].str.replace(abb, description, regex=False)
        
    
     
    print("Tokenizzo")
    tokenized_tweets = []
    for tweet in df_new['text']:
        tknzr = TweetTokenizer()
        tokens = tknzr.tokenize(tweet)
          
        filtered_tweet = []
        
        for word in tokens:
          word = re.sub(r'((.)\2)\2+', r'\1', word.strip().lower())
          if (word.isalnum() or "'" in word):
            if word=="don't":
              word='dont'
            filtered_tweet.append(word)
                
        tokenized_tweets.append(filtered_tweet)

    df_new['filtered'] = tokenized_tweets

    df_new['filtered_text'] = [" ".join(map(str,l)) for l in df_new['filtered']]
    df_new = df_new.drop(columns=['filtered'])
    
    
    #Parsing dates
    df_new['day'] = [date[0] for date in df_new['date'].str.split()]
    df_new['date-month'] = [date[1]+" "+date[2] for date in df_new['date'].str.split()]
    df_new['hour'] = [int(date[3].split(':')[0]) for date in df_new['date'].str.split()]
    
    
    
    if ohe_date is None:
        print("Inizializzo ohe_date")
        ohe_date = OneHotEncoder(sparse=False, dtype=int)
        day_month = ohe_date.fit_transform(df_new[['day','date-month']])
    else:
        day_month = ohe_date.transform(df_new[['day','date-month']])
        
    df_new = df_new.drop(columns=['day','date-month', 'date'])
    df_new = pd.concat([df_new, pd.DataFrame(day_month)], axis=1)
    
    #Add user information
    df_new['filtered_text'] = df_new['filtered_text'] + " " + df['user']    
    
    df_new['length'] = tweet_length
    df_new['contains_links'] = list(map(int, contains_links))
    df_new['contains_mentions'] = list(map(int, contains_mentions))

    
    return df_new, ohe_date

In [ ]:
df = pd.read_csv('development.csv')
cleaned, ohe_date = clean_dataframe(df)
print("duplicates")
ids_to_remove = cleaned[cleaned['ids'].duplicated()]['ids']
cleaned = cleaned[~cleaned['ids'].isin(ids_to_remove)]
final = cleaned.drop(columns=['ids', 'user', 'length'])
final = final.reset_index(drop=True)
y = final['sentiment']
final = final.drop(columns=['sentiment'])

df_eval = pd.read_csv('evaluation.csv')
cleaned_eval, ohe_date = clean_dataframe(df_eval, ohe_date)
cleaned_eval = cleaned_eval.drop(columns=['ids', 'user', 'length'])

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Traduco emoticons
Tokenizzo
Inizializzo ohe_date
duplicates


In [ ]:
params= {
    'ngram_range': [(1,2),(1,3)],
    'max_df': [0.1, 0.2, 0.4, 0.8],
    'min_df': [2, 5, 30]
    }
  
best_config={}
best_f1 = 0.0
for config in ParameterGrid(params):
  print(f"Current configuration: {config}")
  vectorizer = TfidfVectorizer(**config)
  X = vectorizer.fit_transform(cleaned['filtered_text'])
  X = hstack([X, cleaned.iloc[:, 2:]])
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  model = LinearSVC()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  f1 = f1_score(y_test, y_pred, average='macro')
  print(f"F1: {f1}")
  if f1 > best_f1:
    best_f1 = f1
    best_config = config   



Current configuration: {'max_df': 0.4, 'min_df': 2, 'ngram_range': (1, 3)}
F1: 0.8474193509953176


In [ ]:
print(best_config)
print(best_f1)

{'max_df': 0.1, 'min_df': 5, 'ngram_range': (1, 2)}
0.8303882160234863


In [ ]:
X_train_valid, X_test,y_train_valid, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
# interval = np.arange(0.45, 0.65, 0.01)
interval = np.arange(0.1, 0.1, 0.1)
params_svc = {'C': interval,
              'random_state': [42],
              'max_iter': [10000],
              'penalty': ['l1', 'l2']}
f1_scores=[]
kf = KFold(5)
for config in ParameterGrid(params_svc):
  model_f1 = []
  counts = []
  for train_indices, valid_indices in kf.split(X_train_valid):
    X_train = X_train_valid[train_indices]
    y_train = y_train_valid.iloc[train_indices]
    X_valid = X_train_valid[valid_indices]
    y_valid = y_train_valid.iloc[valid_indices]
        
    # keep track of the number of elements in each split
    counts.append(len(train_indices)) 
        
    model = LinearSVC(**config)
    model.fit(X_train, y_train)
    f1 = f1_score(y_valid, model.predict(X_valid), average='macro')
    model_f1.append(f1)
  f1_scores.append(np.average(model_f1, weights=counts))


In [ ]:
f1_scores
list(ParameterGrid(params_svc))[np.argmax(f1_scores)]

{'C': 0.5, 'max_iter': 10000, 'random_state': 42}

In [ ]:
best_config = list(ParameterGrid(params_svc))[np.argmax(f1_scores)]
best_model = LinearSVC(**best_config)
best_model.fit(X_train_valid, y_train_valid)
f1_score(y_test, best_model.predict(X_test), average='macro')

0.844675485197101

In [ ]:
f1_scores_plot = f1_scores
interval_plot = interval
list(ParameterGrid(params_svc))[np.argmax(f1_scores)]
plt.plot(interval[:-1], f1_scores)

In [ ]:
# LINEAR SVC
optimal_config_vect={'max_df': 0.4, 'min_df': 2, 'ngram_range': (1, 3)}
vectorizer = TfidfVectorizer(**optimal_config_vect)
X = vectorizer.fit_transform(final['filtered_text'])
X = hstack([X, final.iloc[:, 2:]])
X_test_eval = vectorizer.transform(cleaned_eval['filtered_text'])
X_test_eval = hstack([X_test_eval, cleaned_eval.iloc[:, 2:]])

optimal_config_model = {'C': 0.5,'max_iter': 10000}
model = LinearSVC(**optimal_config_model)
print("Prediction - evaluation")
model.fit(X, y)
y_pred = model.predict(X_test_eval)
ids = range(0,74999)
subm = pd.DataFrame(y_pred, columns=['Predicted'])
subm['Id'] = ids
subm.to_csv('submission16.csv', index=None)

# COMPLEMENT NAIVE BAYES
optimal_config_vect = {'max_df': 0.1, 'min_df': 5, 'ngram_range': (1, 2)}
vectorizer = TfidfVectorizer(**optimal_config_vect)
X = vectorizer.fit_transform(final['filtered_text'])
X = hstack([X, final.iloc[:, 2:]])
X_test_eval = vectorizer.transform(cleaned_eval['filtered_text'])
X_test_eval = hstack([X_test_eval, cleaned_eval.iloc[:, 2:]])

optimal_config_model = {'alpha': 0.3,'norm': False}
model = ComplementNB(**optimal_config_model)
print("Prediction - evaluation")
model.fit(X, y)
y_pred = model.predict(X_test_eval)
ids = range(0,74999)
subm = pd.DataFrame(y_pred, columns=['Predicted'])
subm['Id'] = ids
subm.to_csv('submission12_new.csv', index=None)

Prediction - evaluation
Prediction - evaluation


In [ ]:
# Wordcloud
index_to_remove = final[final['filtered_text'].str.contains('add everyone')].index
final = final.drop(index_to_remove)
text = " ".join(corpus for corpus in final['filtered_text'])
stopwords = set(STOPWORDS)
# wordcloud = WordCloud(stopwords=stopwords, background_color="white", color_func=color_word).generate(text)
wordcloud = WordCloud(stopwords=stopwords, background_color="white", max_words=180, width=1500, height=1000).generate(text)

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
keys = wordcloud.words_.keys()
substitutions = {'uneasy hesitant': 'uneasy or hesitant',
                 'wink smirk':'wink or smirk'}
positivity = {}
for key in keys:
  #calculate positivity ratio
  if key in substitutions: 
    counts = final[final['filtered_text'].str.contains(substitutions[key])]['sentiment'].value_counts()
  else:
    counts = final[final['filtered_text'].str.contains(key)]['sentiment'].value_counts()

  ratio = counts[1] / counts[0]
  positivity[key] = ratio

#min-max normalization
min_val = min(positivity.values())
max_val = max(positivity.values())

scaled_pos = {k: (v-min_val)/(max_val-min_val) for k,v in positivity.items()}
sorted_words = [k for k, v in sorted(scaled_pos.items(), key=lambda item: item[1])]
sorted_values = sorted(list(scaled_pos.values()))
cmap=plt.get_cmap('seismic')
colors=cmap(sorted_values)

def color_word(word, font_size, position, orientation, random_state=None,**kwargs):
  return matplotlib.colors.rgb2hex(colors[sorted_words.index(word)])

wordcloud = WordCloud(stopwords=stopwords, background_color="white", max_words=180, color_func=color_word, width=1500, height=1000).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud.png',bbox_inches='tight')



In [ ]:
#Initialize dataset
optimal_config_vect={'max_df': 0.4, 'min_df': 2, 'ngram_range': (1, 3)}
vectorizer = TfidfVectorizer(**optimal_config_vect)
X = vectorizer.fit_transform(final['filtered_text'])
X = hstack([X, final.iloc[:, 2:]])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Linear svc, without tuning

In [ ]:
clf = LinearSVC()
clf.fit(X_train, y_train)
print(f"F1:  {f1_score(y_test, clf.predict(X_test), average='macro')}")

F1:  0.8492588607055161


Other classifier, random forest, without tuning

In [ ]:
# Without tuning 0.8437427856013923 in 1.5h
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
print(f"F1:  {f1_score(y_test, clf.predict(X_test), average='macro')}")

In [ ]:
# Without tuning, 0.7892598078310736
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
print(f"F1:  {f1_score(y_test, clf.predict(X_test), average='macro')}")

Other classifier, Complement Naive Bayes.

In [ ]:
# Without tuning
clf = ComplementNB()
clf.fit(X_train, y_train)
print(f"F1:  {f1_score(y_test, clf.predict(X_test), average='macro')}")

F1:  0.8332333210431179


In [ ]:
# Initialize dataset for tuning
X_train_valid, X_test,y_train_valid, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
from sklearn.model_selection import GridSearchCV
interval = np.arange(0.0, 2, 0.1)
params_CNB = {'alpha': interval,
              'norm': [True, False]}
nb = ComplementNB()
clf = GridSearchCV(nb, params_CNB, scoring='f1_macro')
clf.fit(X_train_valid, y_train_valid)
print(f"F1 on test: {f1_score(y_test, clf.predict(X_test), average='macro')}")

F1 on test: 0.83075618194781


In [ ]:
clf.best_params_

{'alpha': 0.30000000000000004, 'norm': False}

In [ ]:
y = df['sentiment']
df = df.drop(columns=['sentiment'])

all_eval = pd.read_csv('evaluation.csv')

In [ ]:
vect_naive = TfidfVectorizer()
X = vect_naive.fit_transform(df['text'])
X_eval = vect_naive.transform(all_eval['text'])
model = LinearSVC()
model.fit(X,y)
y_pred = model.predict(X_eval)
ids = range(0,74999)
subm = pd.DataFrame(y_pred, columns=['Predicted'])
subm['Id'] = ids
subm.to_csv('submission13_svc_naive.csv', index=None)

In [ ]:
from scipy.stats import bernoulli
prob_pos = y.value_counts()[1] / all_develop.shape[0]
y_pred = bernoulli.rvs(prob_pos, size=all_eval.shape[0])
ids = range(0,74999)
subm = pd.DataFrame(y_pred, columns=['Predicted'])
subm['Id'] = ids
subm.to_csv('submission14_random_naive.csv', index=None)